In [1]:
from pyscf import scf,gto
import numpy as np,numpy
from FcMole import FcM

In [2]:
def DeltaV(mol,dL): # dL=[[atmidxs],[atmFcs]]
    mol.set_rinv_orig_(mol.atom_coords()[dL[0][0]])
    dV=mol.intor('int1e_rinv')*dL[1][0]
    for i in range(1,len(dL[0])): 
        mol.set_rinv_orig_(mol.atom_coords()[dL[0][i]])
        dV+=mol.intor('int1e_rinv')*dL[1][i]
    return -dV

In [3]:
#coefficients accurate dL^-8
d1=np.asarray([1/280,-4/105,1/5,-4/5,0,4/5,-1/5,4/105,-1/280])
d2=np.asarray([-1/560,8/315,-1/5,8/5,-205/72,8/5,-1/5,8/315,-1/560])
d3=np.asarray([-7/240,3/10,-169/120,61/30,0,-61/30,169/120,-3/10,7/240])
d4=np.asarray([7/240,-2/5,169/60,-122/15,91/8,-122/15,169/60,-2/5,7/240])
d5=np.asarray([1/6,-3/2,13/3,-29/6,0,29/6,-13/3,3/2,-1/6])
d6=np.asarray([-1/4,3,-13,29,-75/2,29,-13,3,-1/4])

In [4]:
dl=.05
Ps=[]
Es=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
    mf=scf.RHF(fm)
    Es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    Ps.append(mf.make_rdm1())

converged SCF energy = -112.742145890619
converged SCF energy = -112.351169769144
converged SCF energy = -111.967431228079
converged SCF energy = -111.590993296104
converged SCF energy = -111.221914650137
converged SCF energy = -110.860249698719
converged SCF energy = -110.506048710203
converged SCF energy = -110.15935797469
converged SCF energy = -109.820219990832


In [5]:
coeffs=np.array([d1,d2,d3,d4,d5,d6])
coeffs[5]

array([ -0.25,   3.  , -13.  ,  29.  , -37.5 ,  29.  , -13.  ,   3.  ,
        -0.25])

In [6]:
mol0=gto.M(atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
dV=DeltaV(mol0,[[0,1],[1,-1]])
mf0=scf.RHF(mol0)
E0=mf0.scf()
P0=mf0.make_rdm1()

converged SCF energy = -111.221914650137


In [7]:
mf0.energy_nuc()

22.857142857142854

In [8]:
molT=FcM(fcs=[1,-1],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
dV=DeltaV(mol0,[[0,1],[1,-1]])
mfT=scf.RHF(molT)
ET=mfT.scf(dm0=mf.init_guess_by_1e())
PT=mfT.make_rdm1()

converged SCF energy = -105.444757226604


In [9]:
#dE/dZ^(n+1) = \int dP/dZ^n * dV
e=E0+np.einsum('ij,ij',P0,dV) +(mfT.energy_nuc()-mf0.energy_nuc())
print(e)
for i in range (6):
    dP=np.einsum('xij,x->ij',Ps,coeffs[i]/(dl)**(i+1))
    e+=np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2)
    print(e,np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2))

-104.39049415776398
-105.39711356625166 -1.0066194084876745
-105.46687323553378 -0.0697596692821242
-105.43921627227797 0.027656963255811517
-105.44261387741494 -0.0033976051369680347
-105.44451694221489 -0.0019030647999454245
-105.44703462123475 -0.0025176790198576425


In [10]:
#expanding energies
e=E0
print(e)
for i in range (6):
    e+=np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/dl**(i+1)
    print(e,np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/dl**(i+1))

-111.22191465013722
-103.91430431293911 7.307610337198108
-105.39711229982638 -1.4828079868872632
-105.46684776448609 -0.0697354646597148
-105.43920948238166 0.0276382821044289
-105.44316652453043 -0.003957042148764836
-105.44611574647362 -0.0029492219432035592


In [12]:
#to BF
molT=FcM(fcs=[-1,1],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
dV=DeltaV(mol0,[[0,1],[-1,1]])
mfT=scf.RHF(molT)
ET=mfT.scf(dm0=mf.init_guess_by_atom())
PT=mfT.make_rdm1()

converged SCF energy = -119.917957127279


In [13]:
e=E0+np.einsum('ij,ij',P0,dV) +(mfT.energy_nuc()-mf0.energy_nuc())
print(e)
for i in range (6):
    dP=np.einsum('xij,x->ij',Ps,coeffs[i]/(-dl)**(i+1))    # -dl to invert the direction
    e+=np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2)
    print(e,np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2))

-119.00571609489157
-120.01233550338145 -1.0066194084898887
-119.94257583413706 0.06975966924438555
-119.91491887071025 0.027656963426802777
-119.91152126204544 0.003397608664805319
-119.91342433184957 -0.0019030698041282085
-119.91090676889591 0.0025175629536558003


In [14]:
e=E0
print(e)
for i in range (6):
    e+=np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/(-dl)**(i+1)
    print(e,np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/(-dl)**(i+1))

-111.22191465013731
-118.52952498733599 -7.3076103371986845
-120.01233297426869 -1.482807986932699
-119.94259750960482 0.06973546466385963
-119.91495922221573 0.027638287389090497
-119.91100218357231 0.003957038643420673
-119.91395164552105 -0.0029494619487498604


In [15]:
from alch_deriv import alch_deriv

In [18]:
pd1,mo1= alch_deriv(mf0, dL=[0,1],[1,-1])

Failed to parse charges


RuntimeError: No active exception to reraise